## PROJET HADOOP - MS-SIO-2019 - SNCF - API TRANSILIEN

#### SPARK STRUCTURED STREAMING (KAFKA CONSUMER)

P. Hamy,  N. Leclercq, L. Poncet - MS-SIO-2019

In [ ]:
import os
import json
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.sql.window import Window

#### Changement du logging level afin d'éliminer le bruit généré dans la console par un [_warning_](https://stackoverflow.com/questions/39351690/got-interruptedexception-while-executing-word-count-mapreduce-job) récurrent

In [ ]:
log4j = sc._jvm.org.apache.log4j
log4j.LogManager.getRootLogger().setLevel(log4j.Level.ERROR)

#### Création de la session associée au [Strutured Spark Streaming](https://spark.apache.org/docs/latest/structured-streaming-programming-guide.html)

In [ ]:
spark4kafka = SparkSession.builder.appName("MS-SIO-HADOOP-PROJECT-KAFKA-CONSUMER").getOrCreate()

Limitations du nombre de taches de lancées par spark (conseil de configutation glané sur internet pour les configurations matérielles les plus modestes).

In [ ]:
spark4kafka.conf.set('spark.sql.shuffle.partitions', 4)

#### Création du flux Kafka
On utilise ici un 'structured spark stream' associé à une source Kafka. Il s'agit de spécifier la source via l'adresse du serveur Kafka et le nom du topic auquel on souhaite s'abonner. 

In [ ]:
topic = "transilien-02"

In [ ]:
df = spark4kafka \
    .readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "sandbox-hdp.hortonworks.com:6667") \
    .option("subscribe", topic) \
    .option("startingOffsets", "earliest") \
    .load()

#### Schéma de désérialisation des messages  
Les messages injectés dans le flux Kafka sont sérialisés et encodés en binaire dans le champ _value_ du DataFrame _df_.
```
df.printSchema()
root
 |-- key: binary (nullable = true)
 |-- value: binary (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)
 ```
On spécifie le format de désérialisation qui sera passé à la fonction _from_json_. A travers, la variable _json_options_, on précise également le format du champ _timestamp_ afin que les valeurs temporelles soient correctement interprétées.

In [ ]:
schema = StructType(
    [
        StructField("station", IntegerType(), True),
        StructField("timestamp", TimestampType(), True),
        StructField("train", StringType(), True)
    ]
)

In [ ]:
json_options = {"timestampFormat": "yyyy-MM-dd'T'HH:mm:ss.sss'Z'"}

#### Séquence de calcul du temps d'attente moyen par station : étape-01 
Désérialisation/reformatage des messages.

In [ ]:
df0 = df \
    .select(from_json(col("value").cast("string"), schema, json_options).alias("departure")) \
    .select("departure.*")

A l'issue de opération le dataframe _df0_ a le schéma suivant:
```
df0.printSchema()
root
 |-- station: integer (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- train: string (nullable = true)
```
#### Séquence de calcul du temps d'attente moyen par station : étape-02
Spécification de la watermark du stream spark. Nous n'acceptons pas les messages ayant plus d'une minute de retard. Il s'agit d'un choix arbitraire qui n'a que peu d'intérêt dans notre projet. Il toutefois nécessaire de spécifier cette valeur car l'implémentation sous-jacente doit borner l'accumulation des données du stream. [La documentation de Spark explique clairement le concept de _Stateful Stream Processing in Structured Streaming_](https://spark.apache.org/docs/latest/structured-streaming-programming-guide.html).

In [ ]:
df0 = df0.withWatermark('timestamp', '1 minutes')

#### Séquence de calcul du temps d'attente moyen par station : étape-03
Un train apparaitra dans les réponses aux requêtes de l'API SNCF tant que son heure de départ n'appartient pas au passé. On supprime donc les doublons associés aux couples (train, heure de départ). Inutile d'ajouter la station à la contrainte d'exclusion car l'idenfiant d'un train est unique.

In [ ]:
df0 = df0.dropDuplicates(['train', 'timestamp'])

#### Séquence de calcul du temps d'attente moyen par station : étape-04
Définition de la fênêtre (temporelle) dans laquelle les calculs sont aggrégés. Il s'agit ici de calculer le temps d'attente moyen par station sur la dernière heure. Notre fenêtre a donc une largeur temporelle de 60 minutes (_window length_). On choisit de suivre cette moyenne par pas de 2 minutes (_sliding interval_). Dans la mesure où le calcul est demandé par station, la fonction _groupBy_ s'applique au champ _station_ du DataFrame.

In [28]:
df0 = df0.groupBy('station', window('timestamp', '60 minutes', '2 minutes'))

#### Séquence de calcul du temps d'attente moyen par station : étape-05
Pour chaque station, on définit le temps moyen d'attente sur une période _P_ comme le rapport de _P_ vs le nombre de trains au départ de cette station sur la période _P_. Ici _P_ = 60 minutes. On crée une _aggrégation_ sur la fenêtre temporelle précédemment définie. On crée ainsi, pour chaque station et pour chaque fenêtre d'une heure, une colonne _nt_ qui contient le nombre de trains sur la période et une colonne _awt_ qui contiendra le temps d'attente moyen recherché. La colonne _nt_ est donnée à titre indicatif (visualisation dans la console, validation du calcul).

In [ ]:
df0 = df0.agg(count('train').alias('nt'), format_number((60. / count('train')), 2).cast("double").alias('awt'))

#### Séquence de calcul du temps d'attente moyen par station : étape-06
Selection de la fenêtre temporelle associée à la dernière heure. Il s'agit de selectionner, parmis les N fenêtres associées au stream, celle qui correspond à la dernière heure écoulée. On utilise ici la fonction _current_timestamp_ de Spark afin de rendre la sélection dynamique (i.e. glissante). Le calcul est effectué est dans l'unité de _unix_timestamp_ (la seconde) - beaucoup plus facile à manipulée dans ce contexte. Dans l'idée de pourvoir visualiser les valeurs mises en jeu dans le calcul, on choisit de créér les colonnes associées:
- oha = one hour ago = now - 62 minutes = now - (window length + sliding interval) 
- now = now - 1 minutes = now - (sliding interval / 2.) => valeur ajustée pour n'obtenir qu'une seule fenêtre
- wstart = window.start = borne inférieure de la fenêtre temporelle 
- wend = window.wend = borne supérieure de la fenêtre temporelle

La clause _where_ permet de selectionner la fenêtre associée à la dernière heure.  

In [ ]:
df0 = df0 \
    .withColumn('oha', unix_timestamp(current_timestamp()) - 3720) \
    .withColumn('now', unix_timestamp(current_timestamp()) - 60) \
    .withColumn('wstart', unix_timestamp('window.start')) \
    .withColumn('wend', unix_timestamp('window.end')) \
    .where((col('oha') <= col('wstart')) & (col('wend') <= col('now'))) \

#### Séquence de calcul du temps d'attente moyen par station : concaténation des étapes 01 à 06
On s'offre simplement la possibilité d'exécuter la séquence en un seul appel.

In [29]:
df0 = df \
    .select(from_json(col("value").cast("string"), schema, json_options).alias("departure")) \
    .select("departure.*") \
    .select('station', 'train', 'timestamp') \
    .withWatermark('timestamp', '1 minutes') \
    .dropDuplicates(['train', 'timestamp']) \
    .groupBy('station', window('timestamp', '60 minutes', '2 minutes')) \
    .agg(count('train').alias('nt'), format_number((60. / count('train')), 2).cast("double").alias('awt')) \
    .withColumn('oha', unix_timestamp(current_timestamp()) - 3720) \
    .withColumn('now', unix_timestamp(current_timestamp()) - 60) \
    .withColumn('wstart', unix_timestamp('window.start')) \
    .withColumn('wend', unix_timestamp('window.end')) \
    .where((col('oha') <= col('wstart')) & (col('wend') <= col('now'))) \

#### Requête _writeStream_ pour affichage dans la console
Le 'processing' des données est déclenché toutes les minutes (_trigger_). On chosit d'afficher les colonnes suivantes : 
- _station_ : identifiant de la station 
- _window_ : fenêtre temporelle 
- _nt_ : nombre de trains sur la période définie par _window_
- _awt_ : temps d'attente moyen en secondes

In [ ]:
q0 = df0 \
    .select('station', 'window', 'nt', 'awt') \
    .orderBy('station') \
    .writeStream \
    .trigger(processingTime='1 minutes') \
    .outputMode('complete') \
    .format('console') \
    .option('truncate', False) \
    .start()

On obtient ainsi un affichage de la forme : 
```
+--------+------------------------------------------+---+----+
|station |window                                    |nt |awt |
+--------+------------------------------------------+---+----+
|87334482|[2019-02-26 10:16:00, 2019-02-26 11:16:00]|15 |4.0 |
|87366922|[2019-02-26 10:16:00, 2019-02-26 11:16:00]|3  |20.0|
|87381111|[2019-02-26 10:16:00, 2019-02-26 11:16:00]|26 |2.31|
|87381129|[2019-02-26 10:16:00, 2019-02-26 11:16:00]|25 |2.4 |
|87381137|[2019-02-26 10:16:00, 2019-02-26 11:16:00]|39 |1.54|
|87381459|[2019-02-26 10:16:00, 2019-02-26 11:16:00]|24 |2.5 |
|87381657|[2019-02-26 10:16:00, 2019-02-26 11:16:00]|20 |3.0 |
|87381905|[2019-02-26 10:16:00, 2019-02-26 11:16:00]|18 |3.33|
|87382002|[2019-02-26 10:16:00, 2019-02-26 11:16:00]|29 |2.07|
|87382200|[2019-02-26 10:16:00, 2019-02-26 11:16:00]|10 |6.0 |
|87382218|[2019-02-26 10:16:00, 2019-02-26 11:16:00]|26 |2.31|
|87382259|[2019-02-26 10:16:00, 2019-02-26 11:16:00]|10 |6.0 |
|87382267|[2019-02-26 10:16:00, 2019-02-26 11:16:00]|12 |5.0 |
|87382333|[2019-02-26 10:16:00, 2019-02-26 11:16:00]|9  |6.67|
|87382341|[2019-02-26 10:16:00, 2019-02-26 11:16:00]|12 |5.0 |
|87382358|[2019-02-26 10:16:00, 2019-02-26 11:16:00]|24 |2.5 |
|87382366|[2019-02-26 10:16:00, 2019-02-26 11:16:00]|13 |4.62|
|87382374|[2019-02-26 10:16:00, 2019-02-26 11:16:00]|12 |5.0 |
|87382382|[2019-02-26 10:16:00, 2019-02-26 11:16:00]|10 |6.0 |
|87382432|[2019-02-26 10:16:00, 2019-02-26 11:16:00]|13 |4.62|
+--------+------------------------------------------+---+----+```

#### Ecriture du stream sous la forme d'une table Hive 
Idéalement, on souhaiterait utiliser le 'sink Hive' natif que l'on s'attend à trouver dans l'outillage pySpark. Un tel sink n'existe malheureusement pas. [Hortonworks en propose un pour Spark >= 2.3](https://github.com/hortonworks-spark/spark-hive-streaming-sink) mais il s'agit d'une implémentation purement Scala pour laquelle il n'existe pas d'interface Python. On choisit donc de s'orienter vers une solution du type [sink générique](https://docs.databricks.com/spark/latest/structured-streaming/foreach.html) basée sur l'utilisation du callback _streamingDataFrame.writeStream.foreachBatch_.

L'idée est ici de 'transférer' notre DataFrame de la _SparkSession_ de streamming - dont il est issu - vers une _SparkSession_ orientée _Hive_ connectée au _warehouse_ accessible depuis _Tableau_ via le serveur _Spark Thrift_. 

Nous avons intuité cette approche après avoir réalisé un POC d'écriture d'un dataframe Spark sour la forme d'un table visible et accessible depuis _HiveView_ sous _Ambari_.

Le transfert d'une session à l'autre s'effectue dans le contexte d'un callback du type _foreachBatch_
associé à une instance de _StreamingQuery_ (cf. _q1_ ci-dessous). 

Commencons par instancier notre _SparkSession_ connectée à _Hive_...

In [ ]:
spark4hive = SparkSession \
    .builder \
    .master("yarn") \
    .appName("MS-SIO-HADOOP-PROJECT-SPARK-SQL") \
    .config("spark.sql.warehouse.dir", "hdfs://sandbox-hdp.hortonworks.com:8020/api-transilien") \
    .config("hive.metastore.uris", "thrift://sandbox-hdp.hortonworks.com:9083") \
    .enableHiveSupport() \
    .getOrCreate()

#### Création(si besoin) puis sélection de la base 'transilien' sous Hive 

In [ ]:
db_location = "hdfs://sandbox-hdp.hortonworks.com:8020/api-transilien"
spark4hive.sql(f'create database if not exists transilien location "{db_location}"')
spark4hive.sql('use transilien')

#### Implémentation du callback 'foreachBatch'
La fonction _forEachBatchCallback_ sera appelée chaque fois qu'un micro-batch sera délivré par le stream Spark. La période d'appel sera celle du _trigger_ associé à la _StreamingQuery_ (cf _q1_ ci-dessous). Notre solution repose sur l'affectation _spark4hive = streamingdataframe_. Contrairement au comportement attendu dans Python, cet opérateur effectue une copie - ce qui a pour effet de transferer le DataFrame du contexte de _streaming_ vers le contexte _hive_. Il suffit ensuite d'utiliser la fonction _saveAsTable_ en mode _overwrite_ pour écrire le dataframe sous la forme d'une table _Hive_ accéssible sous _Tableau_.

In [ ]:
def forEachBatchCallback(streaming_dataframe, batch_number):
    try:
        spark4hive = streaming_dataframe
        spark4hive.write.mode('overwrite').saveAsTable("averageWaitingTime")
    except Exception as e:
        print(f"failed to update hive table with batch #{batch_number}")
        print(e)

#### Création la _StreamingQuery_ associée au callback _forEachBatchCallback_
En plus de déclencher l'appel à notre callback, cete StreamingQuery nous permet également de sélectionner les champs du DataFrame qui seront injectés dans la table. On choisit ici de ne retenir que l'identifiant de la station et le temps d'attente moyen associé. Les informations relatives à la station - label, coordonnées GPS, ... - seront obtenus par jointure avec une table dédiée disponible sous Hive. 

Note : il aurait été possible d'effectuer cette jointure depuis le stream Spark mais nous choisissons, pour des raisons de simplification, de la réaliser sous Tableau.  

In [ ]:
q1 = df0 \
    .select('station', 'awt') \
    .orderBy('station') \
    .writeStream \
    .trigger(processingTime='1 minutes') \
    .foreachBatch(forEachBatchCallback) \
    .outputMode('complete') \
    .start()

#### Divers 
Les cellules suivantes permettent :
- de stopper les StreamingQuery _q0_ et _q1_  
- de détruire proprement des sessions _spark4kafka_ et _spark4hive_

Note : dans un contexte de production, le code qui précède serait injecté dans un script Python lancé via _spark-submit_. Dans un tel cas, serait nécessaire placer les appels suivants en fin de script : _q0.awaitTermination()_ et _q1.awaitTermination()_

In [ ]:
spark4kafka.streams.active

In [ ]:
q0.stop()
q1.stop()

In [ ]:
spark4kafka.stop()
spark4hive.stop()